# 1. Introduction
Evaluation of derivatives is integral to many machine learning methods. For this purpose, two main methods could be used: symbolical and numerical differentiation. Symbolical differentiation, though straightforward, its implementation requires complex expression manipulation in computer algebra systems, making it very costly to evaluate; the method is also limited to closed-form input expressions. On the other hand, numerical differentiation computes the function derivative by approximating it using small values of step size h; though numerical simpler and faster than symbolic diffferentiation, it suffers from stability issues and round-off or truncation errors.

To address the weaknesses of both these methods, automatic differentiation (AD) was introduced. Since, it has been applied in different areas, such as  engineering design optimization, structural mechanics, and atmospheric sciences; its application to machine learning methods popularised AD. Therefore, due to the important role AD plays in many scientific fields, we introduce a python package that provides user-friendly methods for performing forward-mode AD. Our package supports the evaluation of first derivatives of functions defined by user at given input value. 

# 2. Background
All numerical computation can be seen as a combination of elementary operations for which the derivatives are known. The derivatives of the overall composition can be found by combining the derivatives of elementary operations through the chain rule. Such elementary functions include arithmetic operations (addition, subtraction, multiplication, division), sign switch, and functions such as exponential, logarithm, and the trigonometric (e.g. sin(x), cos(x)). Traces of these elementary operations can be represented by a trace table or a computational graph. Trace table are originally used to trace the value of variables as each line of code is executed. As an example of this flow, Table 1 shows the evaluation trace of elementary operations of the computation f(x<sub>1</sub>) = ln(x<sub>1</sub>) + 3 * x<sub>1</sub> and Figure 1 gives an example of a graphic representation of function f(x<sub>1</sub>) by its elementary operations. 

| Trace       | Elementary function | Current Function Value | Function Derivative |
| ------------- |:-------------:|:-------------:|:-------------:|
| X1      | X1            |  c             | 1  |
| X2      | ln(X1)            | ln(c)      | 1/c |
| X3      | 3 * X1            |  3c        | 3   |
| X4      | X2 + X3             | ln(c) + 3c | 1/c + 3 |

![alt text](sample_trace_graph.png "Figure 1")

The forward mode of AD starts from the input value and compute the derivative of intermediate variables with respect to the input value. Applying the chain rule to each elementary operation in the forward primal trace, we generate the corresponding derivative trace, which gives us the derivative in the final variable. Forward  mode AD can also be viewed as evaluating a function using dual numbers, which are defined as a+b*&epsilon;, where a, b &#1013; R and &epsilon; is a nilpotent number such that &epsilon;^2 = 0 and &epsilon; &ne; 0. It can be shown that the coefficient of &epsilon; after evaluating a function is exactly the derivative of that function, which also works for chain rule.

# 3. Installation
Th package uses a setup.py file for a simple installation, you need to to run the following command in the package main folder:

```
pip install .
```

# 4. How to Use AutoDiff 
In this section, we demo the the AutoDiff package and its capabilities:

In [1]:
import AutoDiff as ad

The AutoDiff package can be used in two different ways:

### 4.1. Pass the function to AD()
You can pass disrectly a \[function, a variable label, and variable value\] to AD as follows:

```
AD(function: str, variable_label:str, variable_value: float): -> (value, derivative)
```

In [8]:
## y = f(x)
y = ad.AD('-x**3 + 2*x**2 + 3*x + 5', 'x', 2)
print("val: ", y.val,"\nder: ", y.der)
print(y)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


In [9]:
## z = f(w)
z = ad.AD('-w**3 + 2*w**2 + 3*w + 5', 'w', 2)
print("val: ", z.val,"\nder: ", z.der)
print(z)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


In [10]:
## Elementary function example  
y = ad.AD('cos(x)', 'x', 0)
print(y)

AD Object: Value = 1.000, Derivative =-0.000


### 4.2. Create an AD_Object()
Alternatively, you can delcare an AD_Object(value) and use it as a variable; the AD object will store the value and derivative with each subsequent elementary operation.  

In [5]:
x = ad.AD_Object(2)
y = -x**3 + 2*x**2 + 3*x + 5
print("val: ", y.val,"\nder: ", y.der)
print(y)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


In [6]:
w = ad.AD_Object(2)
z = -w**3 + 2*w**2 + 3*w + 5
print("val: ", z.val,"\nder: ", z.der)
print(z)

val:  11 
der:  -1
AD Object: Value = 11.000, Derivative =-1.000


In [7]:
## Elementary function example  
x = ad.AD_Object(0)
y = ad.cos(x)
print(y)

AD Object: Value = 1.000, Derivative =-0.000


### 4.3. Supported Elementary Functions
    

The following elementary functions are currently supported by the AutoDiff package:
- addition(+), subtraction(-), multiplication(*) and division(/)
- power (can be called by pow() or using **)
- sine (sin), cosine (cos), tangent (tan)
- natural log (ln), exponential (**note: exp should be called by e() and not exp()**)

We will add more functions as the project proceeds; we envision having most functions implemented in the math module of the standard python library.  

# 5. Software Organization
The package has the following directory structure:
- README.md
- LICENSE.md
- setup.py
- requirements.txt
- test/ (using CodeCov and TravisCI)
    - \__init\__.py
    - test_AutoDiff.py
- docs
    - milestone1.md
    - milestone2.ipynb
    - sample_trace_graph.png
- AutoDiff/
    - \__init\__.py
    - AutoDiff_Class.py

The package uses standard .py standalone files and setup.py for package installation; it will be also distributed through
conda. Finallt, we implemented 31 tests in the test directory; the pytest framework is used, so the tests can be simply run by invoking pytest in that folder. TravisCI and CodeCov are used for integeration and coverage.

# 6. Implementation
Under the **AutoDiff** package, we implemented an Automatic Differentiation Class (*AD*) that stores and tracks the value of the given function and its gradient. This class calls another class, *AD_Object()*, which intitializes the given variable of the function as an automatic differentiation object.  

## 6.1. Current implementation
### 6.1.1. AD_Object class
The *AD_Object* class takes a numeric value as an input and intitializes an *AD* object with the following attributes:
- AD_Object.val: stores the value of the AD Object. 
- AD_Object.der: stores the derivative of the AD Object.

The *AD_Object* class implements all relevant dunder methods(\__repr\__, \__add\__, \__mul\__, etc.) as well as the elementary functions (exp, sin, cos, tan, pow, ln).

Once initialized, the *AD_Object* can be used as a standard variable in python, and with each elementary function applied to it, we return a new *AD_Object* with the updated value and derivative.  

### 6.1.2. AD class
The *AD* class uses the *AD_Object* to initialize AutoDiff objects, but it takes the following three inputs:
1. A user supplied function in string format 
2. The variable label that the function will be derived wrt. 
3. The initial variable value, used for function and derivative evaluation

The main AD class attributes are:
- AD.init_value: initial variable value for evaluation of the function and gradient.
- AD.var_label: variable label to be used (x, y, z, ...etc), for which the derivative of the function is to be returned 
- AD.func_label: the supplied function in string format.
- AD.val: value of evaluated function @ AD.init_value.   
- AD.der: derivative value of supplied function @ AD.init_value.

Finally, the package is designed to have minimal dependencies, with all the elementary functions to be supplied as part of the package; we envision only the *numpy* package will be needed.
## 6.2. To be implemented
At this point, our AutoDiff package accepts only a scaler function of a single input. We are going to implement two additional features for the package:
### 6.2.1. Vectorized input
The package will be extended to accept a vectorized input for points to evaluate the derivative at. Thus for the same function we can return the a vectorized AD.val and AD.der; The accepted vectorized input and saved AD.val and AD.der will be saved in numpy arrays. The code will be extended wth the following pseudo-code: 
```
if isinstance(init_value, np.ndarray):
    AD.val = np.array([AD_object(init).val for init in init_value])
    AD.der = np.array([AD_object(init).der for init in init_value])    
```
### 6.2.2. Multiple input variables
We will extend our package to handle functions of multiple independant variabel e.g f(x,y) and be able te return partial derivatives of such function wrt to each variable. This is still in the preleminary phase, but we will probably modifiy the AD.der attribute to accept the label as an input, so we can call it f.der('x') would give us the derivative wrt 'x', ..etc. The derivative could be installed in a dictionary, where it would be asy to extract the value marked by variable keys. 

### 6.2.3. Elementary functions
As stated earlier, we are still in the process of adding more elementary functions and we envision, by the end of the project, to have added most functions implemented in the math module of the standard python library.  This includes sqrt, tanh, sinh, cosh, ..etc.

# 7. Future Features

### 7.1. Root Finding Algorithm: 
As a showcase application of the AutoDiff package, we will implement a root finding algorithm that can be simply accessed as **AD.root('x')**. As a starter, we will implement the newton-raphson algorithm and supply it with derivateves calulated by our packges; we can later expand and add more methods. Our goal is to have the AutoDiff package for a supplied function, give the user the function value, derivative and roots.     